In [25]:
#Importação das bibliotecas
import requests
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

#Leitura e armazenamento do arquivo bruto

#salarios = pd.read_csv('salaries.csv')
salarios = pd.read_csv('https://raw.githubusercontent.com/Tetzdesen/TP-SAD/main/salaries.csv')

#Tratamento para arquivos duplicados
salarios = salarios.drop_duplicates()

#Pegando valor do Dólar
response = requests.get('https://open.er-api.com/v6/latest/USD')
data = response.json()
cambio = data['rates']['BRL']

#Traduzindo e renomeando
salarios.columns = ['Ano',
                    'Nível de Experiência',
                    'Tipo de Emprego',
                    'Título de Emprego',
                    'Salário',
                    'Moeda',
                    'Salário em Dólar',
                    'Local',
                    'Modalidade',
                    'Local da Empresa',
                    'Tamanho da Empresa']
salarios['Nível de Experiência'] = salarios['Nível de Experiência'].replace({'EX': 'Expert',
                                                                             'SE': 'Sênior',
                                                                             'MI': 'Pleno',
                                                                             'EN': 'Júnior'})
salarios['Tipo de Emprego'] = salarios['Tipo de Emprego'].replace({'PT': 'Meio Período',
                                                                   'FT': 'Tempo Integral',
                                                                   'CT': 'Contrato',
                                                                   'FL': 'Freelance'})
salarios['Tamanho da Empresa'] = salarios['Tamanho da Empresa'].replace({'M': 'Média',
                                                                         'L': 'Grande',
                                                                         'S': 'Pequena'})
salarios['Modalidade'] = salarios['Modalidade'].replace({100: 'Remoto',
                                                         50: 'Híbrido',
                                                         0: 'Presencial'})

#Adicionando colunas novas
salarios['Salário em Reais'] = salarios['Salário em Dólar'] * cambio
salarios['Salário em Reais'] = salarios['Salário em Reais'].astype(int)
salarios['Salário em Reais Mensais'] = (salarios['Salário em Reais']/12).astype(int)

#Formatando
#salarios['Salário em Dólar'] = salarios['Salário em Dólar'].map('$ {:,.2f}'.format)
#salarios['Salário em Reais'] = salarios['Salário em Reais'].map('R$ {:,.2f}'.format)

In [26]:
# Pergunta 1: Meu salário aumenta com o tempo?
fig1 = px.box(
    salarios,
    x='Nível de Experiência',
    y='Salário em Reais Mensais',
    color='Nível de Experiência',
    title='Experiência x Salário',
)
fig1.show()

In [27]:
# Pergunta 2: Escolho entre flexibilidade de horário ou salário?
fig2 = px.box(
    salarios,
    x='Tipo de Emprego',
    y='Salário em Reais Mensais',
    color='Tipo de Emprego',
    points="all",
    title='Tipo de Emprego X Salário',
)
fig2.show()

In [28]:
# Pergunta 3: Se eu sair do país, terei emprego?
mediana_por_local = salarios.groupby('Local da Empresa')['Salário em Dólar'].median().reset_index().round()
mediana_por_local = mediana_por_local.sort_values(by='Salário em Dólar', ascending=False)
mediana_por_local['Salário em Dólar'] = mediana_por_local['Salário em Dólar'].map('$ {:,.2f}'.format)

fig_tabela_mediana = go.Figure(data=[go.Table(
    header=dict(values=['Local de Residência', 'Salário (Mediana)']),
    cells=dict(values=[mediana_por_local['Local da Empresa'], mediana_por_local['Salário em Dólar']]),
)])

fig_tabela_mediana.update_layout(
    title='Países e Suas Medianas Salariais'

)
fig_tabela_mediana.show()

In [29]:
# Pergunta 4: Como é a média salarial de cada nível de experiência a cada ano?
tabela_mediana = salarios.groupby(['Nível de Experiência', 'Ano'])['Salário em Reais Mensais'].median().reset_index().round()
tabela_pivot = tabela_mediana.pivot_table(index='Nível de Experiência', columns='Ano', values='Salário em Reais Mensais').reset_index()

fig = go.Figure(data=[go.Table(
    header=dict(values=['Nível de Experiência'] + list(tabela_pivot.columns[1:])),
    cells=dict(values=[tabela_pivot['Nível de Experiência']] + [tabela_pivot[col].map('R$ {:,.2f}'.format) for col in tabela_pivot.columns[1:]]))
])

fig.update_layout(
    title='Mediana Salarial por Nível de Experiência ao Ano',
)

fig.show()

In [30]:
# Pergunta 5: Se eu optar por trabalhar remotamente, vou receber bem?
media_salarial_por_modalidade = salarios.groupby('Modalidade')['Salário em Reais Mensais'].median().reset_index()

fig = px.pie(
    media_salarial_por_modalidade,
    names='Modalidade',
    values='Salário em Reais Mensais',
    title='Modalidade de Trabalho e Mediana Salarial',
)

fig.show()